#Benchmark of Ministry of Information Technologies and Communications "100 Mil programadores" program

Data about the "100 mil programadores" program, each row is a students or an applicant.

Details of the data can be readed in [Misión TIC 100 mil programadores](https://www.datos.gov.co/Ciencia-Tecnolog-a-e-Innovaci-n/Misi-n-TIC-2020-100-mil-programadores/2emd-i46m)

- Import the libraries to make the benchmark

In [5]:
import requests
import pandas as pd
from fastparquet import write

- Setup and download data

Set "download_data" to True if you want download the data from [Datos Gov](www.datos.gov.co) page directly.

The URL contain the query param to get all the data using the Socrata API (SODA API).
Details about the use of that API can be viewed in:

[SODA endpoints](https://dev.socrata.com/docs/endpoints.html)
[SODA Pagination](https://dev.socrata.com/docs/paging.html)

In [4]:
url_mintic2020_100mil = "https://www.datos.gov.co/resource/2emd-i46m.json?$limit=135000"
download_data = False

if download_data:
    response = requests.get(url = url_mintic2020_100mil)

    #TODO manage error

    data = response.json()

    #Load df as "colums"
    df = pd.DataFrame.from_records(data)

    df.to_csv(r'/home/guille/codes/github/data_benchmark_mintic/raw_data.csv', index = False)
    
else:

    df = pd.read_csv("./raw_data.csv")

df.head()
df.shape

(134563, 15)

In [6]:
write("./data_mintic.parquet", df)

In [7]:
df.describe()

,convocatoria,estrato_social,edad,grado,meta_beneficiario,inversion,meta_dinero
count,134563.0,134563.000000,134563.000000,134563.000000,134563.0,134563.0,1.345630e+05
mean,1.0,2.314522,28.955976,0.184456,5623.0,946611.0,5.322792e+09
std,0.0,0.960925,12.227239,3.440404,0.0,0.0,0.000000e+00
min,1.0,0.000000,0.000000,-1.000000,5623.0,946611.0,5.322792e+09
25%,1.0,2.000000,21.000000,-1.000000,5623.0,946611.0,5.322792e+09
50%,1.0,2.000000,27.000000,-1.000000,5623.0,946611.0,5.322792e+09
75%,1.0,3.000000,36.000000,-1.000000,5623.0,946611.0,5.322792e+09
max,1.0,6.000000,120.000000,11.000000,5623.0,946611.0,5.322792e+09


In [8]:
df.columns

Index(['fecha_corte', 'departame_nombre', 'municipio_nombre',
       'estado_formacion', 'estados', 'convocatoria', 'estrato_social', 'edad',
       'genero', 'ruta', 'grado', 'meta_beneficiario', 'inversion',
       'meta_dinero', 'fecha_vigencia'],
      dtype='object')